In [23]:
import spacy as spacy

nlp = spacy.blank('en')

fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92922860.txt"
strSeparator = '\n#########################################################\n'

In [57]:
a = ('890' 
    + '900')
print(a)

890900


In [ ]:
# first level of shortening
from spacy.tokens import Span
from spacy.matcher import Matcher

nlp = spacy.blank("en")
with open(fName, encoding="utf8") as f:
    text = f.read()

doc = nlp(text)

matcher = Matcher(nlp.vocab)

#matcher.add('TOORGINCOUNTRY', [pattern2])

pattern3 = [
    {'LOWER':'CONTRIBUTING'.lower()},
    {'LOWER':'TO'.lower()}, 
    {'LOWER':'YOUR'.lower()}, 
    {'LOWER':'SUCCESS'.lower()}
    ]
matcher.add('ENDPAGE', [pattern3])
'''Unit
Article
code
Price
EUR
Unit
Amount
EUR'''
pattern4 = [
    {'LOWER':'article'},
    {'LOWER':'code'}, 
    {'LOWER':'price'}, 
    {'LOWER':'eur'},
    {'LOWER':'unit'},
    {'LOWER':'amount'},
    {'LOWER':'eur'}
]
matcher.add('STARTPAGE', [pattern4])
'''STAUFF
L.L.C.
Doc.no
.
/
Date
Page'''
pattern5  = [
    {'LOWER':'doc.no'},
    {'LOWER':'.'},
    {'LOWER':'/'},
    {'LOWER':'date'},
    {'LOWER':'page'}
]
 
pattern6  = [
    {'LOWER':'doc.по'},
    {'LOWER':'.'},
    {'LOWER':'/'},
    {'LOWER':'date'},
    {'LOWER':'page'}
] 
matcher.add('ABOUTNOPAGE', [pattern5, pattern6])

doc = nlp(text)

matches = matcher(doc)
spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]

paging = []

pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
shortenText = ''
firstPage = True
txt = '''import spacy
from spacy.tokens import Span

nlp = spacy.blank("en")\n'''
docs  = []
npos = 0
for sp in spans:
    if sp.label_  == 'ABOUTNOPAGE':
        pageinfo[sp.label_] = sp.end
    elif sp.label_  == 'STARTPAGE':
        pageinfo[sp.label_] = sp.end
    elif sp.label_  == 'ENDPAGE':
        pageinfo[sp.label_] = sp.start
        paging.append(pageinfo)
        
        if firstPage ==True:
            shortenText = doc[:pageinfo['ENDPAGE']].text
            firstPage = False
            docs.append(doc[:pageinfo['ENDPAGE']].text)
            txt = txt + strSeparator + doc[:pageinfo['ENDPAGE']].text 
        else:
            shortenText = shortenText +  doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text
            docs.append(doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text)
            print(pageinfo, pageinfo['ENDPAGE'] - pageinfo['STARTPAGE'])
            txt = txt + strSeparator + doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text 

            
            #break
        pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}

    #print(pageinfo)
#print(txt)
#print(len(paging))


with open(fName.replace('.txt', 's.txt'), encoding="utf8", mode='w') as f:
    f.write(txt)


In [ ]:
dds = nlp('Export - Customs tariff no.: 84812010')
for sp in dds:
    print(sp)

In [71]:
rtest = '''Purchase order number: N SR-1-06 1043
1800 5 PC 9910000425 2,00 1 РС 10,00

CAT-5-QUICK-RELEASE CPLNGS-ENG
Kat-5-Schnellverschlusskupplungen-ENG
packed per each item

Product description: sales catalogue
Export - Customs tariff по.: 49111010
Country of origin: Germany'''

#with open(fName.replace('.txt', 's.txt'), encoding="utf8") as f:
    #rtest = f.read()

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)
patternTariff = [{'LOWER': 'customs'}, 
    {'LOWER':'tariff'}, 
    {'IS_ALPHA': True},
    {'IS_PUNCT': True, 'OP': '*'}, 
    #{'LOWER': '.'},
    #{'LOWER': ':'}, 
    {'IS_DIGIT': True}]
matcher.add('PTRTARIFF', [patternTariff])

patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True},
    {'LOWER' : {'in' : ['pc', 'рс']}},
    {'LIKE_NUM':True}    ] 

matcher.add('PTROFPC', [patternPC])

doc1 = nlp(rtest)
print("11:", [token.text for token in doc1[5:24]])
matches = matcher(doc1)
spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

for sp in spans:
    print(sp)

11: ['SR-1', '-', '06', '1043', '\n', '1800', '5', 'PC', '9910000425', '2,00', '1', 'РС', '10,00', '\n\n', 'CAT-5', '-', 'QUICK', '-', 'RELEASE']
1800 5 PC 9910000425
2,00 1 РС 10,00
Customs tariff по.: 49111010


In [ ]:
rtest = '''Purchase order number: N SR-1-06 1015
1700 7 РС 9910000425 2,00 1 РС 14,00

CAT-5-QUICK-RELEASE CPLNGS-ENG
Kat-5-Schnellverschlusskupplungen-ENG
packed per each item

Product description: sales catalogue
Export - Customs tariff по.: 49111010
Country of origin: Germany'''

In [89]:
with open(fName.replace('.txt', 's.txt'), encoding="utf8") as f:
    rtest = f.read()
rtest = rtest.replace(strSeparator, '')

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)

patternPurchase = [{'LOWER': 'purchase'}, 
    {'LOWER':'order'}, 
    {'LOWER':'number'}, 
    {'LOWER': ':'}]
matcher.add('PTRPURCHASE', [patternPurchase])

patternTariff = [{'LOWER': 'customs'}, 
    {'LOWER':'tariff'}, 
    {'IS_ALPHA': True},
    {'IS_PUNCT': True, 'OP': '*'}, 
    {'IS_DIGIT': True}]
matcher.add('PTRTARIFF', [patternTariff])

patternCountry = [{'LOWER': 'country'}, 
    {'LOWER':'of'} , 
    {'LOWER': 'origin'}, 
    {'LOWER': ':'}, 
    {'IS_ALPHA': True}]
matcher.add('PTRCOUNTRY', [patternCountry])

patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True},
    {'LOWER' : {'in' : ['pc', 'рс']}},
    {'LIKE_NUM':True}    ] 

matcher.add('PTROFPC', [patternPC])

doc1 = nlp(rtest)

#print("Verbs:", [token.text for token in doc1[45:54]])

matches = matcher(doc1)
spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

startSpan = 0
endSpan = 0
posTariff = 0
txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
nPos = 0
pos1PC = 0
pos2PC = 0
for sp in spans:
    if sp.label_  == 'PTRPURCHASE':
        startSpan = sp.start
        #print('here is a purchase', sp.start, doc1[sp.start: sp.end]) 
    elif sp.label_  == 'PTRTARIFF':
        posTariff = sp.end-1
        #print('here is a tariff', sp.end, doc1[sp.end-1: sp.end]) 
    elif sp.label_  == 'PTROFPC':
        pos2PC = sp.start
        #print('here is a summa', sp.end, doc1[sp.start: sp.end]) 

    elif sp.label_  == 'PTRCOUNTRY':
        endSpan = sp.end
        pos2PC = pos2PC - startSpan  
        #print(doc1[startSpan + pos2PC], doc1[startSpan + pos2PC +1 ],doc1[startSpan + pos2PC+2],doc1[startSpan + pos2PC+3],            doc1[startSpan + pos2PC+4])  
        #break
        #print(doc1[startSpan+pos2PC + 4].text )
        sDoc = "doc" + str(nPos)
        txt2 = txt2 + '\n\n' + sDoc + " = nlp('''" + doc1[startSpan:endSpan].text + "''')\n" 
        #txt2 = ''
        txt2 = (txt2 + '#' + sDoc + str(doc1[startSpan + 5 : startSpan + 8].text + ' '
            + doc1[startSpan + 8].text + ' ' + doc1[startSpan + 10].text + ' ' + doc1[startSpan + 11].text + ' '
            + doc1[startSpan+13].text + ' ' + doc1[startSpan+pos2PC].text + ' ' + doc1[startSpan+pos2PC + 1].text  + ' '
            + doc1[startSpan+pos2PC + 4].text + ' ' +  doc1[posTariff].text + ' ' + doc1[endSpan - 1].text).replace('\n', '') + '\n')
        #print(txt2)
        #break
        #txt2 = txt2 + sDoc + ".ents = [Span(" + sDoc + ', 3, 4, label="GPE")] \n'
        txt2 = txt2 + '''print(''' + '"' + sDoc + '",' + sDoc + "[5: 8], " + sDoc + "[8: 9], " + sDoc + "[10: 11], " + sDoc +"[11: 12], " 
        txt2 = txt2 +  sDoc + "[13: 14], " + sDoc + "["+str(pos2PC) + ':' + str(pos2PC + 1) + "], "
        txt2 = txt2 +  sDoc + "[" +str(pos2PC+1) + ':' + str(pos2PC + 2) + "] ," + sDoc + "["+str(pos2PC + 3) + ':' + str(pos2PC + 4) + "], " + sDoc + "["+ str(posTariff - startSpan)+ ': ' + str(posTariff - startSpan + 1) + "], " + sDoc + "[" + str(endSpan - startSpan - 1)+ ': ' + str(endSpan - startSpan) + "]) \n"
        txt2 = txt2 +  sDoc + ".ents = [Span(" + sDoc + ', 5, 8, label="CONTRACT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 8, 9, label="CONTRACT1"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 10, 11, label="POS"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 11, 12, label="AMOUNT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', 13, 14, label="ARTICLE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC) + ', ' + str(pos2PC + 1) + ', label="PRICE"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 1) + ', ' + str(pos2PC + 2) + ', label="UNIT"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(pos2PC + 3) + ', ' + str(pos2PC + 4) + ', label="SUM"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(posTariff - startSpan)+ ', ' + str(posTariff - startSpan + 1) + ', label="TARIFF"), \n'
        txt2 = txt2 +  '    Span(' + sDoc + ', ' + str(endSpan - startSpan - 1)+ ', ' + str(endSpan - startSpan) + ', label="COUNTRY")] \n'

        txt2 = txt2 + 'docs.append(' + sDoc + ')\n'
        nPos += 1
        #break
#print(txt2)
#exit

txt2 = txt2 + 'doc_bin = DocBin(docs = docs)\n'
txt2 = txt2 + "doc_bin.to_disk('./train.spacy')"
with open(fName.replace('.txt', '.py'), encoding="utf8", mode='w') as f:
    f.write(txt2)

In [93]:
#%run -i "F:\Python\OCR_diploma\docs_base\StauffInvoices\92922860.py"
%run -i "92922860.py"

doc0 SR-1-06 604 1200 4 6100074594 13,79 1 55,16 84812010 Germany
doc1 SR-1-06 711 1300 50 6010003476 237,81 100 118,91 73079910 Germany
doc2 SR-1-06 923 1400 100 6100049574 1,96 1 196,00 73079910 Germany
doc3 SR-1-06 933 1500 300 6100049574 1,96 1 588,00 73079910 Germany
doc4 SR-1-06 942 1600 481 6100076765 3,85 1 1.851,85 84812010 Germany
doc5 SR-1-06 1015 1700 7 9910000425 2,00 1 14,00 49111010 Germany
doc6 SR-1-06 1043 1800 5 9910000425 2,00 1 10,00 49111010 Germany
doc7 SR-1-06 1178 1900 2 6100232260 140,25 1 280,50 84212980 Germany
doc8 SR-1-06 1231 2500 40 6010003710 393,09 100 157,24 73079910 Germany
doc9 SR-1-06 1267 2600 1 6100122987 0,49 1 0,49 39235090 Italy
doc10 SR-1-06 1362 2700 400 2012031318 0,25 1 100,00 73079980 Germany
doc11 SR-1-06 1432 2800 474 6100079873 8,49 1 4.024,26 84212980 Italy
doc12 SR-1-06 1426 2900 12 6010000239 715,29 100 85,83 84813091 Germany
doc13 SR-1-06 1448 3000 40 6010003710 393,09 100 157,24 73079910 Germany
doc14 SR-1-06 1457 3100 1 6100068507

In [ ]:
python -m spacy train ./config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy

In [37]:
import random

d = [1,2,3,4,5,6,7]

random.shuffle(d)
print(d)

[7, 6, 3, 4, 2, 1, 5]


In [30]:
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 604
1200 4 РС 6100074594 (*) 13,79 1 РС 55,16

QRC-FF-19-M-22L-BP-W3

FF19-2-L2230

packed per each item

Export - Customs tariff no.: 84812010
Country of origin: Germany''')
#doc0SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
print("doc0",doc0[5: 8], doc0[8: 9], doc0[10: 11], doc0[11: 12], doc0[13: 14], doc0[17:18], doc0[18:19] ,doc0[19:20], doc0[50: 51], doc0[56: 57]) 
doc0.ents = [Span(doc0, 5, 8, label="CONTRACT"), 
    Span(doc0, 8, 9, label="CONTRACT1"), 
    Span(doc0, 10, 11, label="POS"), 
    Span(doc0, 11, 12, label="AMOUNT"), 
    Span(doc0, 13, 14, label="ARTICLE"), 
    Span(doc0, 17, 18, label="PRICE"), 
    Span(doc0, 18, 19, label="UNIT"), 
    Span(doc0, 19, 20, label="SUM"), 
    Span(doc0, 50, 51, label="TARIFF"), 
    Span(doc0, 56, 57, label="COUNTRY")] 
docs.append(doc0)


doc1 = nlp('''Purchase order number: N SR-1-06 711
1300 50 PC 6010003476 (*) 237,81 100 PC 118,91

FI-EWD-15L-V-W3-DKO
FI-EWD-15L-B-W3-DKO

packed per each item

Product description: fitting

Export - Customs tariff no.: 73079910
Country of origin: Germany''')
#doc1SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany
print("doc1",doc1[5: 8], doc1[8: 9], doc1[10: 11], doc1[11: 12], doc1[13: 14], doc1[17:18], doc1[18:19] ,doc1[19:20], doc1[59: 60], doc1[65: 66]) 
doc1.ents = [Span(doc1, 5, 8, label="CONTRACT"), 
    Span(doc1, 8, 9, label="CONTRACT1"), 
    Span(doc1, 10, 11, label="POS"), 
    Span(doc1, 11, 12, label="AMOUNT"), 
    Span(doc1, 13, 14, label="ARTICLE"), 
    Span(doc1, 17, 18, label="PRICE"), 
    Span(doc1, 18, 19, label="UNIT"), 
    Span(doc1, 19, 20, label="SUM"), 
    Span(doc1, 59, 60, label="TARIFF"), 
    Span(doc1, 65, 66, label="COUNTRY")] 
docs.append(doc1)


#print(doc1)


doc0 SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
doc1 SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany


In [24]:
import json 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span, DocBin 
docs = [] 


doc0 = nlp('''Purchase order number: N SR-1-06 604
1200 4 РС 6100074594 (*) 13,79 1 РС 55,16

QRC-FF-19-M-22L-BP-W3

FF19-2-L2230

packed per each item

Export - Customs tariff no.: 84812010
Country of origin: Germany''')
#doc0SR-1-06 604 1200 4 6100074594 13,79 1 РС 84812010 Germany
print("doc0",doc0[5: 8], doc0[8: 9], doc0[10: 11], doc0[11: 12], doc0[13: 14], doc0[17:18], doc0[18:19] ,doc0[19:20], doc0[50: 51], doc0[56: 57]) 
doc0.ents = [Span(doc0, 5, 8, label="CONTRACT"), 
    Span(doc0, 8, 9, label="CONTRACT1"), 
    Span(doc0, 10, 11, label="POS"), 
    Span(doc0, 11, 12, label="AMOUNT"), 
    Span(doc0, 13, 14, label="ARTICLE"), 
    Span(doc0, 17, 18, label="PRICE"), 
    Span(doc0, 18, 19, label="UNIT"), 
    Span(doc0, 19, 20, label="SUM"), 
    Span(doc0, 50, 51, label="TARIFF"), 
    Span(doc0, 56, 57, label="COUNTRY")] 
docs.append[doc0]


doc1 = nlp('''Purchase order number: N SR-1-06 711
1300 50 PC 6010003476 (*) 237,81 100 PC 118,91

FI-EWD-15L-V-W3-DKO
FI-EWD-15L-B-W3-DKO

packed per each item

Product description: fitting

Export - Customs tariff no.: 73079910
Country of origin: Germany''')
#doc1SR-1-06 711 1300 50 6010003476 237,81 100 PC 73079910 Germany
print("doc1",doc1[5: 8], doc1[8: 9], doc1[10: 11], doc1[11: 12], doc1[13: 14], doc1[17:18], doc1[18:19] ,doc1[19:20], doc1[59: 60], doc1[65: 66]) 
doc1.ents = [Span(doc1, 5, 8, label="CONTRACT"), 
    Span(doc1, 8, 9, label="CONTRACT1"), 
    Span(doc1, 10, 11, label="POS"), 
    Span(doc1, 11, 12, label="AMOUNT"), 
    Span(doc1, 13, 14, label="ARTICLE"), 
    Span(doc1, 17, 18, label="PRICE"), 
    Span(doc1, 18, 19, label="UNIT"), 
    Span(doc1, 19, 20, label="SUM"), 
    Span(doc1, 59, 60, label="TARIFF"), 
    Span(doc1, 65, 66, label="COUNTRY")] 
docs.append[doc1]


SR-1-06 604 1200 4 6100074594 13,79 1 55,16 84812010 Germany


In [61]:
pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
pageinfo['ABOUTNOPAGE'] = 10

print(pageinfo)

a = dict()
a['21'] = 10
a['22'] =100
print(a)

{'ABOUTNOPAGE': 10, 'STARTPAGE': 0, 'ENDPAGE': 0}
{'21': 10, '22': 100}


In [ ]:
doc1 = nlp('Purchase order number: 6100074594 ( * )')
doc1.ents = [Span(doc1, 4, 5, label='Article')]

doc2 = nlp('Country of origin: Germany')
doc2.ents = [Span(doc1, 4, 5, label='Country')]

In [ ]:
startPos = 483
for i, token in enumerate(doc[startPos: startPos + 300]):
    print(token)

In [41]:
doc1.text

'Purchase order number: N SR-1-06 711\n1300 50 PC 6010003476 (*) 237,81 100 PC 118,91\n\nFI-EWD-15L-V-W3-DKO\nFI-EWD-15L-B-W3-DKO\n\npacked per each item\n\nProduct description: fitting\n\nExport - Customs tariff no.: 73079910\nCountry of origin: Germany'

In [98]:
nlp_ = spacy.load(r"C:\Users\AdminAsus\source\python\spacyST\spacySt-1\output\model-best")
docx = nlp_(doc191.text)
for ent in docx.ents:
    print(ent.text, ent.label_)

SR-1-06 CONTRACT
1870 CONTRACT1
20800 POS
900 AMOUNT
6100152347 ARTICLE
12,73 PRICE
100 UNIT
114,57 SUM
73181692 TARIFF
Germany COUNTRY
